In [1]:
import json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import pickle
from itertools import combinations

import pprint
import glob
from random import random, randrange
from random import choice
import logging
from collections import defaultdict
import sys

import subprocess
import shlex
from scipy.stats import pearsonr

import pandas as pd

import datetime
from datetime import datetime, timedelta
from dateutil.parser import parse
from collections import Counter
from scipy.signal import savgol_filter
import itertools
import time
import csv

import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import trange, tqdm

from sklearn.decomposition import LatentDirichletAllocation,NMF

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\u1085188\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\u1085188\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
start_time = time.time()

In [3]:
# Using the regular dictionary
f = open('pruned_retweet_dict','rb')
retweet_dict = pickle.load(f)
f.close()

# We now assign communities using the saved data
PT = nx.read_gexf('PT-pruned.gexf')

True


In [4]:
datetime.strptime(retweet_dict[1212147705507700736]['date'],'%y-%m-%d-%H:%M:%S')

datetime.datetime(2019, 12, 31, 23, 11, 5)

In [5]:
retweet_dict[1212147705507700736]['date']

'19-12-31-23:11:05'

In [6]:
# clusters we care about
clusters = [13,48,6,49,104]
cluster_names = ['Democrats',"Republicans","Unorthodox","Public Health","Antivaxxers"]

In [7]:
def author_to_cluster(author):
    return(PT.nodes[author]['louvain'])

In [8]:
author_to_cluster('LotusOak2')

104

In [9]:
mininglist1 = ['vax', 'vaxxed', 'vaccine', 'vaccination', 'vaccinations', 
'vaxsafety', 'vax saftey', 'vaccineswork', 'vaccines work', 'vaccinesaftey',
'vaccine saftey', 'vaccines revealed', 'vaccinesrevealed', 
'novax', 'no vax', 'no-vax', 'antivax', 'anti-vax', 'anti vax', 'immunisation', 
'Vaccin', 'Vaccinaties', 'vaccinatiezorg', 'vaccine injury', 'vax injury',
'vaccinatieschade']

mininglist2 = ['\#vax', '\#vaxxed', '\#vaccine', '\#vaccination', '\#vaccinations', '\#vaxsafety', '\#vaccineswork', '\#vaccinesaftey', '\#vaccinesrevealed', '\#novax', '\#antivax', '\#immunisation', '\#Vaccin', '\#Vaccinaties', '\#vaccinatiezorg', 
'\#vaccinatieschade', '\#nvkp', '\#rvp', '\#rijksvaccinatieprogramma', '\#vaccineinjury', 
'\#vaxinjury', '\#anti-vax']

extralist = ['vaccines','coronavirus','corona','covid19','virus','vaccine','covid','covid-19',\
             'covidー19','#covidー19','vaccin','im','covid19','would',\
            'dont', 'year', 'going', 'months', 'first', 'new']

In [10]:
import re

# Def Remove emoji from string
def deEmojify(s):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',s)


# Def remove arabic
# According to Wikipedia, Arabic characters fall in the Unicode range 0600 - 06FF. 
def remove_foreign(s): 
    arabic_list = re.findall(r'[\u0600-\u06FF]+',s)
    for w in arabic_list:
        s = s.replace(w, " ")
        
    chinese_list = re.findall(r'[\u4e00-\u9FFF]+',s)
    for w in chinese_list:
        s = s.replace(w, " ")
        
    hindi_list = re.findall(r'[\u0900-\u097F]+',s)
    for w in hindi_list:
        s = s.replace(w, " ")
             
    korean_list = re.findall(r'[\u3040-\u30ff]+',s)
    for w in korean_list:
        s = s.replace(w, " ")
              
    japanese_list = re.findall(r'[\u3040-\u30ff]+',s)
    for w in japanese_list:
        s = s.replace(w, " ")
        
    return(s)


def light_preprocessor(s):   
    s = s.strip().lower()
    s = s.replace('\n'," ")
    s = s.replace('-', " ")
    s = s.replace('.', "")
    # Remove hashtags
    s = s.replace('#',"")
    # Remove Emojis
    s = deEmojify(s)
    # Remove foreign
    s = remove_foreign(s)
    
    s = re.sub("\s+"," ",s)
    
    s = [x for x in s if (x.isalnum() or x == " " or x=="#" or x=="@")]
    s = "".join(s)
    # (Dont) Remove stopwords in english and french
    # stop_words = stopwords.words('english') + stopwords.words('french')
    s = " ".join([x for x in s.split() if (#(x not in stop_words) and
                                           (x.find('http') == -1) 
                                           #and (x[0] != '#')
                                           and (x[0] != '@') #remove @user
                                          and (x != 'amp')
                                         and (x not in mininglist1)
                                        and (x not in mininglist2)
                                    and (x not in extralist)
                                          and not(x.isnumeric()))]) # there might be some redundancies with the first line
    s = s.replace('/',' ')
    
    return s

In [11]:
# Try processor over n random tweets
def try_processor(n=10):
    for i in range(n):
        random_tweet = choice(list(retweet_dict.values()))
        processed_tweet = light_preprocessor(random_tweet['text'])
        print("This is the unprocessed tweet: " + str(random_tweet['text']))
        print("This is the processed tweet: " + str(processed_tweet))

In [12]:
try_processor()

This is the unprocessed tweet: Premier essai clinique d'un vaccin contre le coronavirus

https://t.co/fBNOzeDkBf https://t.co/pXfjiIJuBR
This is the processed tweet: premier essai clinique dun contre le
This is the unprocessed tweet: @forwardarc @realDonaldTrump It'll be China that finds and mass produces a vaccine, then billionaire Jack Ma will buy and donate 300 million doses to the American people and we'll all become pro-China and Alibaba customers for life.  It'll be the biggest PR coup in history.
This is the processed tweet: itll be china that finds and mass produces a then billionaire jack ma will buy and donate million doses to the american people and well all become pro china and alibaba customers for life itll be the biggest pr coup in history
This is the unprocessed tweet: .@moderna_tx , a biotech company based in Cambridge, Mass., has shipped the first batches of its #COVID-19 vaccine. The vaccine was created 42 days after the genetic sequence of the COVID_19 virus was rel

In [13]:
create_list = False

In [14]:
if create_list:
    corpus_list = dict()
    for c in clusters:
        corpus_list[str(c)]=[]

    for t in tqdm(retweet_dict):
        a = retweet_dict[t]['author']
        try:
            c = author_to_cluster(a)
            if c in clusters:
                text = light_preprocessor(retweet_dict[t]['text'])
                corpus_list[str(c)].append(text)
        except KeyError:
            pass

In [15]:
def create_corpus(date1,date2):
    date1 = datetime.strptime(date1,"%d/%m/%y")
    date2 = datetime.strptime(date2,"%d/%m/%y")
    corpus = dict()
    for c in clusters:
        corpus[str(c)]=""

    for t in tqdm(retweet_dict):
        a = retweet_dict[t]['author']
        try:
            c = author_to_cluster(a)
            if c in clusters:
                rt_date = datetime.strptime(retweet_dict[t]['date'],'%y-%m-%d-%H:%M:%S')
                if (date1 <= rt_date) and (rt_date <= date2):
                    text = " " + light_preprocessor(retweet_dict[t]['text'])
                    corpus[str(c)] += text
        except KeyError:
            pass
    return corpus

In [16]:
# Testing
test = create_corpus('26/12/19','28/12/19')
test

{'13': ' the us food and drug administration has approved for the time in the united states a for the prevention of the deadly ebola the agency announced the search for an hiv may soon be over thats it a single dose of hpv may protect against infection a study says this could mean a lot for worldwide coverage rates four beauties visited us recently for puppies foxnews spreads anti vaxxer propaganda measles is not a big deal to me at all measles is a highly contagious airborne and can sometimes be fatal but is highly preventable through weak minded maga love this stuff mog this one was important i reported that a systemic flaw in the way the associated press website was functioning resulted in anti conspiracy theories being published with ap branding which disinfo actors used to their advantage quick facts work we evolved from filthy monkey men climate change is real earth isnt flat religion is cancer gmos are safe everything you eat is a chemical healthcare should be universal public e

In [17]:
# clusters we care about
clusters = [13,48,6,49,104,57,85,80]
cluster_names = ['Democrats',"Maga","Socialists","Health","Antivaxers","French","Indian","FreedomUK"]

complete_corpus = create_corpus('26/12/19','27/05/20')
pre_corpus = create_corpus('26/12/19','11/03/20')
post_corpus = create_corpus('11/03/20','27/05/20')

for i in range(len(clusters)):
    text_file = open(str(cluster_names[i]) + "_full" + ".txt", "w",encoding='utf8')
    text_file.write(complete_corpus[str(clusters[i])])
    text_file.close()
    text_file = open(str(cluster_names[i]) + "_pre" + ".txt", "w",encoding='utf8')
    text_file.write(pre_corpus[str(clusters[i])])
    text_file.close()
    text_file = open(str(cluster_names[i]) + "_post" + ".txt", "w",encoding='utf8')
    text_file.write(post_corpus[str(clusters[i])])
    text_file.close()


In [18]:
elapsed_time = time.time() - start_time
print(elapsed_time)

35987.710688114166
